In [ ]:
import joblib

import numpy as np
import networkx as nx

from src.CBN import CausalBayesianNetwork as CBN
import modularised_utils as mut
import Linear_Additive_Noise_Models as lanm
import operations as ops
import evaluation_utils as evut
import params

### Define low-level DCM "LL" and high-level DCM "HL"

In [2]:
experiment = 'synth1'

In [3]:
S = 'Smoking'
T = 'Tar'
C = 'Cancer'

S_ = 'Smoking_'
C_ = 'Cancer_'

In [4]:
ll_endogenous_coeff_dict = {(S, T): 0.3, (T, C): 0.2}
ll_causal_graph          = CBN(list(ll_endogenous_coeff_dict.keys()))

hl_endogenous_coeff_dict = {(S_, C_): 0.0}
hl_causal_graph          = CBN(list(hl_endogenous_coeff_dict.keys()))

In [5]:
num_llsamples   = params.n_samples[experiment][0]
num_hlsamples   = params.n_samples[experiment][1]

### Construct the empirical nominal distribution for the low-level model

In [6]:
ll_mu_hat    = np.array([0, 0, 0])  
ll_Sigma_hat = np.diag([1, 1, 1])  

hl_mu_hat    = np.array([0, 0]) 
hl_Sigma_hat = np.diag([1, 1]) 

### Define the sets of relevant interventions and the (total) surjective and order-preserving function $ω:I^{L} \mapsto I^{H}$

In [7]:
iota0 = None
iota1 = ops.Intervention({S:0})
iota2 = ops.Intervention({S:0, T:1})
iota3 = ops.Intervention({S:1})
iota4 = ops.Intervention({S:1, T:0})
iota5 = ops.Intervention({S:1, T:1})

eta0 = None
eta1 = ops.Intervention({S_:0})
eta2 = ops.Intervention({S_:1})

omega = {   
            iota0: eta0,
            iota1: eta1,
            iota2: eta1,
            iota3: eta2,
            iota4: eta2,
            iota5: eta2
        }

Ill_relevant = list(set(omega.keys()))
Ihl_relevant = list(set(omega.values()))

### Sampling and Pair construction

In [8]:
Dll_samples, Dll_noise = {}, {}
for iota in Ill_relevant:

    llcm              = lanm.LinearAddSCM(ll_causal_graph, ll_endogenous_coeff_dict, iota)
    lenv_iota         = mut.sample_distros_Gelbrich([(ll_mu_hat, ll_Sigma_hat)])[0] 
    Dll_noise[iota]   = lenv_iota.sample(num_llsamples)[0]
    Dll_samples[iota] = llcm.simulate(Dll_noise[iota], iota)

### Define the abstraction T

In [9]:
T = np.array([[1, 2, 1], [0, 1, 0]])

### Compute the empirical nominal distribution for the high-level model and the linear coefficients

In [10]:
data_observational_hl         = Dll_samples[None]@ T.T
hl_endogenous_coeff_dict      = mut.get_coefficients(data_observational_hl, hl_causal_graph) 
U_hl, hl_mu_hat, hl_Sigma_hat = mut.lan_abduction(data_observational_hl, hl_causal_graph, hl_endogenous_coeff_dict)

### Generate samples for the high-level model

In [12]:
Dhl_samples, Dhl_noise = {}, {}
for eta in Ihl_relevant:

    if eta is not None:
        hlcm             = lanm.LinearAddSCM(hl_causal_graph, hl_endogenous_coeff_dict, eta)
        lenv_eta         = mut.sample_distros_Gelbrich([(hl_mu_hat, hl_Sigma_hat)])[0] 
        Dhl_noise[eta]   = lenv_eta.sample(num_hlsamples)[0]
        Dhl_samples[eta] = hlcm.simulate(Dhl_noise[eta], eta)

    else:
        Dhl_noise[eta]   = U_hl
        Dhl_samples[eta] = data_observational_hl

In [14]:
Ds = {}
for iota in Ill_relevant:
    Ds[iota] = (Dll_samples[iota], Dhl_samples[omega[iota]])

### Save the data

In [ ]:
joblib.dump((ll_causal_graph, Ill_relevant), f"data/{experiment}/LL.pkl")
joblib.dump(ll_endogenous_coeff_dict, f"data/{experiment}/ll_coeffs.pkl")

joblib.dump((hl_causal_graph, Ihl_relevant), f"data/{experiment}/HL.pkl")
joblib.dump(hl_endogenous_coeff_dict, f"data/{experiment}/hl_coeffs.pkl")

joblib.dump(Ds, f"data/{experiment}/Ds.pkl")

joblib.dump(T, f"data/{experiment}/Tau.pkl")
joblib.dump(omega, f"data/{experiment}/omega.pkl")

joblib.dump((Dll_noise[None], ll_mu_hat, ll_Sigma_hat), f"data/{experiment}/exogenous_LL.pkl")
joblib.dump((U_hl, hl_mu_hat, hl_Sigma_hat), f"data/{experiment}/exogenous_HL.pkl")